# Autoencoder 2
This is from https://github.com/L1aoXingyu/pytorch-beginner/tree/master/08-AutoEncoder
conv_autoencoder.py

In [25]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [26]:
if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

In [27]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [28]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [29]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [30]:
dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [31]:
class autoencoder(nn.Module):
    def __init__(self):
        # Q: How to understand the input-output dimensions in the CNN tranformation.
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [32]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

In [35]:
for epoch in range(num_epochs):
    loss_total = 0
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===
        loss_total += loss
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss_total))
#     if epoch % 10 == 0:
#         pic = to_img(output.cpu().data)
#         save_image(pic, './dc_img/image_{}.png'.format(epoch))

# torch.save(model.state_dict(), './conv_autoencoder.pth')

epoch [1/100], loss:128.7307
epoch [2/100], loss:93.7665
epoch [3/100], loss:69.7349
epoch [4/100], loss:64.4504
epoch [5/100], loss:61.3336
epoch [6/100], loss:59.3160
epoch [7/100], loss:57.7742
epoch [8/100], loss:56.5276
epoch [9/100], loss:55.3596
epoch [10/100], loss:54.3362
epoch [11/100], loss:53.4044
epoch [12/100], loss:52.5901
epoch [13/100], loss:51.8883
epoch [14/100], loss:51.2620
epoch [15/100], loss:50.6982
epoch [16/100], loss:50.1725
epoch [17/100], loss:49.7230
epoch [18/100], loss:49.3538
epoch [19/100], loss:48.9873
epoch [20/100], loss:48.7311
epoch [21/100], loss:48.4676
epoch [22/100], loss:48.1537
epoch [23/100], loss:47.9315
epoch [24/100], loss:47.6903
epoch [25/100], loss:47.4492
epoch [26/100], loss:47.3162
epoch [27/100], loss:47.0777
epoch [28/100], loss:46.9033
epoch [29/100], loss:46.7459
epoch [30/100], loss:46.5810
epoch [31/100], loss:46.3980
epoch [32/100], loss:46.2933
epoch [33/100], loss:46.1335
epoch [34/100], loss:45.9993
epoch [35/100], loss:4

KeyboardInterrupt: 